# Ansible Einführung

Warum eigentlich Ansible und nicht bash-Skripte?

In welchem Verzeichnis befinden wir uns?

In [ ]:
pwd

##  Vorbereitung der Container

Zunächst alle weg werfen und neu aufsetzen.

In [ ]:
for i in ais-bashy db web1 web2 playbooks; do
    sudo lxc-destroy --name ${i} --force
done
ansible-playbook -i localhost, ../00-home-base/setup-lxc.yml
export AIS_BASHY_IP=$(sudo lxc-ls ais-bashy --fancy | awk '{ print $5 }' | grep '.\.')
echo $AIS_BASHY_IP
ssh root@${AIS_BASHY_IP} uptime

## Inhalt des Verzeichnisses

Falls `tree` nicht installiert sein sollte, dann installieren und nochmal ausführen.

In [ ]:
tree . || (sudo apt-get update && sudo apt-get install --yes tree && tree .)

## Das Shell Skript

In [ ]:
cat install-nginx.sh

Wegen `set -euo pipefail` siehe

https://vaneyckt.io/posts/safer_bash_scripts_with_set_euxo_pipefail/

Wegen `cd "${BASH_SOURCE%/*}" || exit` siehe

http://mywiki.wooledge.org/BashFAQ/028

In [ ]:
ssh root@${AIS_BASHY_IP} install-nginx.sh

Damit wir das Skript ausführen können, muss dieses auf das Zielsystem (target) kopiert werden. __Darüber hinaus__ auch die Konfigurationsdateien usw.

In [ ]:
scp -r install-nginx.sh config/ root@${AIS_BASHY_IP}:/tmp/

In [ ]:
ssh root@${AIS_BASHY_IP} ls /tmp/

In [ ]:
ssh root@${AIS_BASHY_IP} /tmp/install-nginx.sh

In [ ]:
echo https://${AIS_BASHY_IP}

Das Skript ist noch nicht ganz richtig, da wir ein systemd System verwenden.

In [ ]:
tail -n 8 install-nginx.sh

In [ ]:
sed -rie 's/^(service.*$)/# \1/g' install-nginx.sh
sed -rie 's/^# (systemctl.*)/\1/g' install-nginx.sh

In [ ]:
tail -n 8 install-nginx.sh

In [ ]:
scp -r install-nginx.sh config/ root@${AIS_BASHY_IP}:/tmp/

In [ ]:
ssh root@${AIS_BASHY_IP} /tmp/install-nginx.sh

Das zwiemalige Ausführen eines Bash-Skriptes kann zu einem anderen Ergebnis führen, als beim ersten mal.

_Hinweis:_ Unsere Änderung betrifft nicht das entfernen der nginx config Datei.

In [ ]:
cat Bash-Problems

In [ ]:
sed -rie 's/^# (service.*$)/\1/g' install-nginx.sh
sed -rie 's/^(systemctl.*)/# \1/g' install-nginx.sh
tail -n 8 install-nginx.sh

##  Nginx mit Ansible aufsetzen

Zunächst den Container neu aufsetzen.

In [ ]:
sudo lxc-destroy --name ais-bashy --force

Die Container einrichten und die Umgebungsvariable setzen.

In [ ]:
ansible-playbook -i localhost, ../00-home-base/setup-lxc.yml
export AIS_BASHY_IP=$(sudo lxc-ls ais-bashy --fancy | awk '{ print $5 }' | grep '.\.')
echo $AIS_BASHY_IP
ssh root@${AIS_BASHY_IP} uptime

In [ ]:
sudo lxc-ls --fancy

In [ ]:
ansible-playbook -i $AIS_BASHY_IP, -u root install-nginx.yml

In [ ]:
cat install-nginx.yml

In [ ]:
echo http://$AIS_BASHY_IP

In [ ]:
ansible-playbook -i $AIS_BASHY_IP, -u root install-nginx.yml

Das mehrmalige ausführen eines Ansible-Skripts führt zum gleichem Ergebnis. Dies nennt man Idempotenz.

Siehe auch: https://de.wikipedia.org/wiki/Idempotenz